# Logistic Regression - Human Activity Recognition

En éste ejercicio vamos a reutilizar los resultados obtenidos con PCA para el dataset de Human Activity Recognition e intentar clasificar correctamente las muestras que denotan movimiento de las que no. Para más información sobre los datos, consultar la fuente [original](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones).

### Importamos las librerías

In [ ]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook

%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

RANDOM_STATE = 17

### Cargamos el dataset

In [ ]:
PATH_DATASET = "/content"

In [ ]:
X_train = np.loadtxt(os.path.join(PATH_DATASET, "X_train.txt"), delimiter='\t', dtype=str).astype(float)
y_train = np.loadtxt(os.path.join(PATH_DATASET, "y_train.txt"), delimiter='\t', dtype=str).astype(int)

### EDA Mínimo

In [ ]:
print("Estructura features del dataset: {}".format(X_train.shape))
print("Estructura de las etiquetas: {}".format(y_train.shape))

Estructura features del dataset: (7352,)
Estructura de las etiquetas: (7352,)


In [ ]:
# Exploramos un poco las features
df = pd.DataFrame(data=X_train)
df.describe()

,0
count,7352
unique,7352
top,2.8858451e-001 -2.0294171e-002 -1.3290514e-0...
freq,1


In [ ]:
df.isna().sum().sum()

0

In [ ]:
# Cantidad única de clases
clases = np.unique(y_train)
clases

array([1, 2, 3, 4, 5, 6])

In [ ]:
n_clases = clases.size

In [ ]:
n_clases

6

|     Label     |   Descripción  | 
| ------------- |:-------------: |
|       1       | Caminar        | 
|       2       | Subir escaleras|
|       3       | Bajar escaleras|
|       4       | Estar sentado  |
|       5       | Estar parado   |
|       6       | Recostarse     |

### Aplicamos PCA

In [ ]:
# Estandarizamos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

ValueError: ignored

Al aplicar PCA, es una práctica común reducir el número de dimensiones, dejando tantos componentes como sean necesarios para que contemplen al menos el 90% de la varianza de los datos escalados originales. Scikit-learn permite indicar directamente ese parámetro como condición. Si no se puede seleccionar un número alto de componentes y mediante un scree plot, verificar cuántos son necesarios para mantener un 90%.

In [ ]:
pca = PCA(n_components=0.9, random_state=RANDOM_STATE).fit(X_scaled)
X_pca = pca.transform(X_scaled)

In [ ]:
# Cantidad de componentes necesarios
X_pca.shape[1]

In [ ]:
pca_2 = PCA(n_components=200, random_state=RANDOM_STATE).fit(X_scaled)

plt.figure(figsize=(10,7))
plt.plot(np.cumsum(pca_2.explained_variance_ratio_), color='k', lw=2)
plt.xlabel('Cantidad de componentes')
plt.ylabel('Total varianza contemplada')
plt.xlim(0, 200)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.axvline(63, c='b')
plt.axhline(0.9, c='r')
plt.show();

In [ ]:
# Graficamos los primeros dos componentes
plt.figure(figsize=(12,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_train, edgecolor='none', alpha=0.7, s=40, cmap=plt.cm.get_cmap('nipy_spectral', 6))
plt.colorbar()
plt.title('HAR - PCA projection 2D');

### Clusterización

In [ ]:
kmeans = KMeans(n_clusters=n_clases, n_init=100, random_state=RANDOM_STATE)
kmeans.fit(X_pca)
cluster_labels = kmeans.labels_

In [ ]:
# Graficamos los primeros dos componentes - clusters id
plt.figure(figsize=(12,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels, edgecolor='none', alpha=0.7, s=40, cmap=plt.cm.get_cmap('nipy_spectral', 6))
plt.colorbar()
plt.title('HAR - PCA projection 2D');

In [ ]:
kmeans_move = KMeans(n_clusters=2, n_init=100, random_state=RANDOM_STATE)
kmeans_move.fit(X_pca)
cluster_labels_mov = kmeans_move.labels_

In [ ]:
# Graficamos los primeros dos componentes - clusters id
plt.figure(figsize=(12,10))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=cluster_labels_mov, edgecolor='none', alpha=0.7, s=40, cmap=plt.cm.get_cmap('nipy_spectral', 2))
plt.colorbar()
plt.title('HAR - PCA projection 2D');

In [ ]:
tab = pd.crosstab(y_train, cluster_labels, margins=True)
tab.index = ['Caminar', 'Subir escaleras', 'Bajar escaleras', 'Estar parado', 'Estar sentado', 'Recostarse', 'Todos']
tab.columns = ['cluster ' + str(i + 0) for i in range(6)] + ['Todos']
tab

In [ ]:
tab = pd.crosstab(y_train, cluster_labels_mov, margins=True)
tab.index = ['Caminar', 'Subir escaleras', 'Bajar escaleras', 'Estar parado', 'Estar sentado', 'Recostarse', 'Todos']
tab.columns = ['cluster ' + str(i + 0) for i in range(2)] + ['Todos']
tab

---

## Clasificación

Se puede observar en el gráfico de los primeros dos componentes de PCA, y en los resultados de la clusterización por K-means con dos clusters, que potencialmente se podrían separar con los dos primeros componentes, los estados de movimiento respecto de los estados en reposo. 

In [ ]:
y_train[0:5]

In [ ]:
# Armamos máscaras para mapear las actividades a reposo (1) o movimiento (0)
# y = {1, 2, 3} => 0, y = {4, 5, 6} => 1 

act_mask = y_train <= 3
st_mask = y_train > 3

In [ ]:
st_mask[:5]

In [ ]:
y_class_train = y_train.copy()

y_class_train[act_mask] = 0
y_class_train[st_mask] = 1

In [ ]:
y_class_train[:5]

In [ ]:
y_class_train.shape

In [ ]:
x_class_train = X_pca[:, :2]

In [ ]:
x_class_train.shape

### Creamos el Modelo

In [ ]:
import sys  
sys.path.insert(0, 'C:/Users/Lautaro/PycharmProjects/ceia_intro_a_IA/clase_3/ejercicios/src')

In [ ]:
from models import BaseModel
from metrics import Accuracy, Precision, Recall

In [ ]:
class LogisticRegression(BaseModel):
    
    # definimos la función sigmoid para entrenamiento y las predicciones
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    # definimos la función loss para reportarla cada cierta cantidad de epochs
    def loss(self, y, y_hat):
        loss = np.mean(-y*(np.log(y_hat)) - (1-y)*np.log(1-y_hat))
        return loss 

    def fit(self, X, y, lr, b, epochs, bias=True, log=100, verbose=True):

        # si decidimos utilizar bias, agregamos como siempre una columna con '1' al dataset de entrada
        if bias:
            X = np.hstack((np.ones((X.shape[0], 1)), X))

        # inicializamos aleatoriamente los pesos
        m = X.shape[1]
        W = np.random.randn(m).reshape(m, 1)

        loss_list = []
        
        # corremos Mini-Batch para optimizar los parámetros
        for j in range(epochs):
            idx = np.random.permutation(X.shape[0])
            X_train = X[idx]
            y_train = y[idx]
            batch_size = int(len(X_train) / b)

            for i in range(0, len(X_train), batch_size):
                end = i + batch_size if i + batch_size <= len(X_train) else len(X_train)
                batch_X = X_train[i: end]
                batch_y = y_train[i: end]

                prediction = self.sigmoid(np.sum(np.transpose(W) * batch_X, axis=1))
                error = prediction.reshape(-1, 1) - batch_y.reshape(-1, 1)
                grad_sum = np.sum(error * batch_X, axis=0)
                grad_mul = 1 / batch_size * grad_sum
                gradient = np.transpose(grad_mul).reshape(-1, 1)

                W = W - (lr * gradient)
            
            l_epoch = self.loss(y_train, self.sigmoid(np.dot(X_train, W)))
            loss_list.append(l_epoch)
            if verbose:
                if j%log==0:
                    print("Epoch: {}, Loss: {}".format(j, l_epoch))
                
        self.model = W
        self.losses = loss_list

    def predict(self, X):
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        p = self.sigmoid(X @ self.model)
        mask_true = p >= 0.5
        mask_false = p < 0.5
        p[mask_true] = 1
        p[mask_false] = 0
        return p

In [ ]:
# Seleccionar hiperparámetros
lr = 0.01
b = 16
epochs = 10000
bias = True

# Hacer el fit del modelo con los HPs seleccionados
logistic_regression = LogisticRegression()
logistic_regression.fit(x_class_train, y_class_train.reshape(-1, 1), lr, b, epochs, bias, log=500)
print(logistic_regression.model)

### Decision Boundary para Dataset de Train

In [ ]:
# Calculamos el slope e intercept para graficar
slope = -(logistic_regression.model[1] / logistic_regression.model[2])
intercept = -(logistic_regression.model[0] / logistic_regression.model[2])

In [ ]:
# Graficos

plt.scatter(x_class_train[:, 0], x_class_train[:, 1], c=y_class_train[:])
y_vals = intercept + (slope * x_class_train[:, 0])
plt.plot(x_class_train[:, 0], y_vals, c="k")
plt.title('Decision Boundary')
plt.ylim((-30, 60))
plt.show()

### Predicciones sobre el Dataset de Test

In [ ]:
# Cargamos el dataset de test

PATH_DATASET = "data/UCI HAR Dataset/test"

X_test = np.loadtxt(os.path.join(PATH_DATASET, "X_test.txt"))
y_test = np.loadtxt(os.path.join(PATH_DATASET, "y_test.txt")).astype(int)

In [ ]:
X_test.shape

In [ ]:
# Debemos aplicar las mismas transformaciones sobre el dataset de test que se
# realizaron al dataset de train

X_test_scaled = scaler.transform(X_test)
X_test_pca = pca.transform(X_test_scaled)
x_class_test = X_test_pca[:, :2]

In [ ]:
x_class_test.shape

In [ ]:
# Armamos máscaras para mapear las actividades a reposo (1) o movimiento (0)
# y = {1, 2, 3} => 0, y = {4, 5, 6} => 1 

act_mask = y_test <= 3
st_mask = y_test > 3

In [ ]:
y_class_test = y_test.copy()

y_class_test[act_mask] = 0
y_class_test[st_mask] = 1

In [ ]:
y_class_test.shape

In [ ]:
predictions = logistic_regression.predict(x_class_test)

### Calculamos las Métricas

In [ ]:
metrics = [Accuracy(), Precision(), Recall()]
results = {}
for metric in metrics:
    name = metric.__class__.__name__
    results[name] = metric(y_class_test, predictions[:, 0])
    print('{metric}: {value}'.format(metric=name, value=results[name]))

### Graficamos los resultados

In [ ]:
# Graficos

f, (ax, bx) = plt.subplots(2, 1, sharey='col', figsize=(15, 15))

ax.scatter(x_class_test[:, 0], x_class_test[:, 1], c=y_class_test)
ax.set_title('Valores reales')
ax.set_ylim((-30, 60))

bx.scatter(x_class_test[:, 0], x_class_test[:, 1], c=predictions[:, 0])
y_vals = intercept + (slope * x_class_test[:, 0])
bx.plot(x_class_test[:, 0], y_vals, c="k")
bx.set_title('Predicciones')
bx.set_ylim((-30, 60))

plt.show()

---

### Bibliografía

* [Información sobre el dataset](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones)